In [1]:
import torch
print(torch.__version__)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

2.2.0


'cuda:0'

定义参数

In [61]:
model_name="facebook/nllb-200-distilled-1.3B"
source_lang="en"
target_lang="de"
dataset_name="wmt16"

下载读取WMT数据集

In [41]:
from datasets import load_dataset

# Download the WMT dataset
dataset = load_dataset(dataset_name, f"de-en")
translation=dataset["test"]["translation"][:100]
source_texts = [t[source_lang] for t in translation]  # Adjust the size as needed
reference_texts = [t[target_lang] for t in translation]

Found cached dataset wmt16 (C:/Users/tiantian/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


  0%|          | 0/3 [00:00<?, ?it/s]

测试模型

In [63]:
from transformers import pipeline
lang_piece=dict(src_lang='eng_Latn',
    tgt_lang='deu_Latn')
translation_pipeline = pipeline(task=f"translation", model=model_name,device=device)
translation_pipeline('hello world', **lang_piece)

KeyboardInterrupt: 

生成翻译句子

In [ ]:
import pandas as pd
translations = translation_pipeline(source_texts, **lang_piece)
translations = [translation['translation_text'] for translation in translations]
# 输出翻译结果
df = pd.DataFrame({
    'Source': source_texts,
    'Translation': translations
})
df

使用sacrebleu评价模型

In [57]:
import sacrebleu
bleu_score = sacrebleu.corpus_bleu(translations, [reference_texts]).score
f"BLEU Score: {bleu_score}"

'BLEU Score: 44.75055716583148'

写入文件

In [58]:
with open("./BLEU_result.txt","a",encoding="utf-8") as f:
    f.write(f"\n{model_name} {source_lang} {target_lang} {bleu_score}")